In [ ]:
import datetime
import os
from collections import defaultdict
from pathlib import Path

In [ ]:
servers = ["webhost2"]
possible_logs = ["C:\\inetpub\\logs\\LogFiles", "E:\\inetpub\\logs\\LogFiles"]

In [ ]:
def convertPathtoUnc(server, path):
    drive = path[0] + "$"

    converted_path = path[3:]  # cut off the drive letter x://

    out = "\\\\{server}\{drive}\{converted_path}".format(
        server=server, drive=drive, converted_path=converted_path
    )

    return out

In [ ]:
def getSites(server):
    for pl in possible_logs:
        try:
            for file in os.listdir(convertPathtoUnc(server, pl)):
                file = os.path.join(pl, file)
                yield (Path(file).name)
        except:
            pass

In [ ]:
list(getSites("inside"))

In [ ]:
def getLogs(server, site):
    for log_path in possible_logs:
        try:
            today = datetime.datetime.now().date()
            for file in os.listdir(
                convertPathtoUnc(server, os.path.join(log_path, site))
            ):
                file = os.path.join(log_path, site, file)
                unc_path = convertPathtoUnc(server, file)
                date_modified = datetime.datetime.fromtimestamp(
                    os.path.getmtime(unc_path)
                ).date()
                if date_modified == today:
                    yield (unc_path)
        except Exception as e:
            print(e)

In [ ]:
list(getLogs("webhost2", "W3SVC10"))

In [ ]:
def RequestedFile(log):
    headers = [
        "date",
        "time",
        "s-ip",
        "cs-method",
        "cs-uri-stem",
        "cs-uri-query",
        "s-port",
        "cs-username",
        "c-ip",
        "cs(User-Agent)",
        "sc-referrer",
        "sc-status",
        "sc-substatus",
        "sc-win32-status",
        "sc-bytes",
        "cs-bytes",
        "time-taken",
    ]

    with open(log, "r") as f:
        for line in f.readlines():
            line = line.strip()
            if line.startswith("#"):
                continue
            data = line.split()
            yield dict(zip(headers, data))

In [ ]:
def PrintRequestedFiles(log):
    with open(log, "r") as f:
        for line in f.readlines():
            line = line.strip()
            if line.startswith("#"):
                continue
            data = line.split(" ")
            print(data)


def YieldRequestedFiles(log):
    with open(log, "r") as f:
        for line in f.readlines():
            line = line.strip()
            if line.startswith("#"):
                continue
            data = line.split(" ")
            yield (data)

In [ ]:
def RequestedFiles(log):
    ips = defaultdict(
        lambda: defaultdict(
            lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(int)))
        )
    )

    with open(log, "r") as f:
        for line in f.readlines():
            line = line.strip()
            if line.startswith("#"):
                continue
            data = line.split(" ")
            print(data)
            client_ip = data[10]
            file = data[6]
            method = data[5]
            status_code = data[14]
            user_agent = data[11]
            ips[client_ip][user_agent][method][status_code][file] += 1
    return ips

In [ ]:
def RequestedFilesSimple(log):
    ips = defaultdict(lambda: defaultdict(int))
    ua = defaultdict(lambda: defaultdict(int))
    with open(log, "r") as f:
        for line in f.readlines():
            line = line.strip()
            if line.startswith("#"):
                continue
            data = line.split(" ")
            client_ip = data[10]
            file = data[6]
            method = data[5]
            status_code = data[14]
            site = data[13]
            user_agent = data[11]
            ips[client_ip][file] += 1
            ua[client_ip][user_agent] += 1
    return dict(ips), dict(ua)

In [ ]:
from pprint import pprint

from ipwhois import IPWhois


def getOwner(ip, extra=False):
    obj = IPWhois(ip)
    results = obj.lookup_rdap(depth=1)
    if extra:
        pprint(results)
    return results["asn_description"]


# getOwner("3.134.114.180",extra=True)

In [ ]:
import ipaddress
import json

aws_ips = "https://ip-ranges.amazonaws.com/ip-ranges.json"
import requests

aws_ips = requests.get(aws_ips).json()

aws_map = {}
for region in aws_ips["prefixes"]:
    if "us-" in region["region"]:
        aws_map[region["ip_prefix"]] = region["region"]


with open("BlockedRanges.json", "w") as f:
    f.write(json.dumps(list(aws_map.keys()), indent=4))


def checkAws(ip):
    for r in aws_map:
        if ipaddress.IPv4Address(ip) in ipaddress.IPv4Network(r):
            return aws_map[r]

In [ ]:
server = "webhost2"
who = defaultdict(list)
f = 0
for service in getSites(server):
    for log_path in getLogs(server, service):
        f = 0
        print(log_path, end=": ")
        try:
            for row in YieldRequestedFiles(log_path):
                if "/sheriff/Print1.aspx" in row:
                    f += 1
                    who[row[10]].append(row[7])
            print(f"found {f}")
        except Exception as e:
            print(e)

for ip in who:
    o = "Private"
    try:
        o = getOwner(ip)
    except Exception as e:
        pass
    print(ip, o)

In [ ]:
server = "webhost2"
who = defaultdict(list)
f = 0
for service in getSites(server):
    for log_path in getLogs(server, service):
        f = 0
        print(log_path, end=": ")
        try:
            for row in YieldRequestedFiles(log_path):
                if any(["/ViewInmate/" in s for s in row]):
                    f += 1
                    who[row[10]].append(row[6])
            print(f"found {f}")
        except Exception as e:
            print(e)

for ip in who:
    if len(who[ip]) > 200:
        o = "Private"
        try:
            o = getOwner(ip)
        except Exception as e:
            print(e)
            pass
        print(ip, o, len(who[ip]))

In [ ]:
# get users who access inside server
seen = []
server = "webhost2"
for service in getSites(server):
    for log_path in getLogs(server, service):
        data = RequestedFile(log_path)
        for item in data:
            if item["cs-username"] not in seen:
                seen.append(item["cs-username"])
                print(item["cs-username"])

In [ ]:
for ip in who:
    tot = sum(data[ip].values())
    try:
        owner = checkAws(ip)
        if tot > 1000:
            if owner is None:
                owner = getOwner(ip)
            print(ip, " = ", tot, "Requests : owned by", owner)
            for u in uas[ip]:
                print(u.strip())
            print()
    except Exception as e:
        print(e)

In [ ]:
for server in servers:
    for site in getSites(server):
        print(site)
        for log in getLogs(server, site):
            ip_data = RequestedFiles(log)
            for ip in ip_data:
                print(ip)
                for user_agent in ip_data[ip]:
                    print(user_agent)
                    for method in ip_data[ip][user_agent]:
                        print("\t", method)
                        for status_code in ip_data[ip][user_agent][method]:
                            print("\t\t", status_code)
                            for file in ip_data[ip][user_agent][method][status_code]:
                                print(
                                    "\t\t\t",
                                    ip_data[ip][user_agent][method][status_code][file],
                                    file,
                                )
                    print()
            print()
            print()
        print("#" * 50)